In [31]:
#import package
import bs4
import requests
import pandas as pd
import geopandas as gpd
import random
import math
from geopy.distance import geodesic
import sqlalchemy as db
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import matplotlib

## Part1 Data Processing

## distance calculation
Define a function to caculate distcance between two pairs of given longtitude and latitude

In [2]:
#define a function to calculate distance
def create_distance(lat1,lon1,lat2,lon2):
    dist = geodesic((lat1,lon1),(lat2,lon2)).km
    return dist

## uber_date
1. Import and go through the data, delete the data out of given range
2. Crate a 'year' column to obatin the # of datapoints in ervery year
3. Drop invalid variables and add distance to the dataframe

In [3]:
#import uber_data
uber = pd.read_csv('uber_rides_sample.csv')

In [4]:
#check
uber.head(2)

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1


In [5]:
# keep data in required range
uber_select = uber[(uber['pickup_longitude'] >= -74.242330) & (uber['pickup_longitude'] <= -73.717047) \
    & (uber['pickup_latitude'] >= 40.560445) & (uber['pickup_latitude'] <= 40.908524) &\
    (uber['dropoff_longitude'] >= -74.242330) & (uber['dropoff_longitude'] <= -73.717047) \
    & (uber['dropoff_latitude'] >= 40.560445) & (uber['dropoff_latitude'] <= 40.908524)]

In [6]:
#will sample certain number of data from yellow taxi data based on this
uber_select['year'] = pd.to_datetime(uber['pickup_datetime']).dt.year
uber_select['year'].value_counts()

<ipython-input-6-77b182171db7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_select['year'] = pd.to_datetime(uber['pickup_datetime']).dt.year


2012    31531
2011    31102
2013    30496
2009    30077
2010    29494
2014    29258
2015    13514
Name: year, dtype: int64

In [7]:
#drop invalid variables and add distance to the dataframe
uber_select.drop(['Unnamed: 0','key','fare_amount','passenger_count', 'year'], axis = 1, inplace = True)

<ipython-input-7-4a960f229f9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_select.drop(['Unnamed: 0','key','fare_amount','passenger_count', 'year'], axis = 1, inplace = True)


In [8]:
uber_select['distance'] = uber_select.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)

<ipython-input-8-355f9ddcbd83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_select['distance'] = uber_select.apply(lambda x: create_distance(x['pickup_latitude'],\


In [9]:
uber_data = uber_select.reset_index()
uber_data = uber_data.drop(['index'], axis = 1)

In [10]:
uber_data['pickup_datetime'] = pd.to_datetime(uber_data['pickup_datetime'])

In [11]:
uber_data.head(2)

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1.681111
1,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,2.454363


## yellow_taxi data
1. Improt taxi_zone.shp and convert geometry data into normal longtitue and lantitude
2. Get the url of yellow taxi and collect the data as yearly file and check the data
3. Drop invalid variables
4. Drop the data with same PULocationID and DOLocationID, merge geographical information and drop data out of given range
5. Sample the data accoring to the data amouont in uber
6. Add distance to the dataframe
7. Because data in 2010 and 2009 have different information in it, we process them seperately

In [12]:
#import taxi_zone information
taxi_zone = gpd.read_file('taxi_zones.shp')

In [13]:
#check
taxi_zone.head(2)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."


In [14]:
#change geometry information to lantitude and longtitde
taxi_zone = taxi_zone.to_crs(4326)
taxi_zone['lon'] = taxi_zone.centroid.x  
taxi_zone['lat'] = taxi_zone.centroid.y

<ipython-input-14-770757aa3575>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone['lon'] = taxi_zone.centroid.x
<ipython-input-14-770757aa3575>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone['lat'] = taxi_zone.centroid.y


In [15]:
taxi_zone.head(2)

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,lon,lat
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",-74.174000,40.691831
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745


In [16]:
#get the url of yellow taxi
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"


def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html


def find_taxi_parquet_links():
    html = get_taxi_html()
    soup = bs4.BeautifulSoup(html, 'html.parser')
    yellow_lst = [a['href'] for a in soup.find_all(title = 'Yellow Taxi Trip Records')]
    return yellow_lst

yellow_lst =  find_taxi_parquet_links()


In [17]:
#create the year_num list according to uber_data 
year_num = [13514, 29258, 30496, 31531, 31102, 29494, 30077]

In [18]:
#define the function to get and clean data in 2011-2015
def get_and_clean_data(n):
    file = list(range(1,13))
    for i in range(len(file)):
        file[i] = pd.read_parquet(yellow_lst[i+12*n-84])
    for i in range(len(file)):
        file[i].drop(['VendorID','tpep_dropoff_datetime','passenger_count',\
                            'trip_distance','RatecodeID','store_and_fwd_flag',\
                            'payment_type','fare_amount','extra','mta_tax',\
                            'tolls_amount','improvement_surcharge','total_amount',
                            'congestion_surcharge','airport_fee'], \
                           axis = 1, inplace = True)
    for i in range(len(file)):
        file[i].drop(file[i][file[i]['PULocationID'] == file[i]['DOLocationID']].index, inplace = True)
    for i in range(len(file)):
        file[i].rename(columns = {'PULocationID' : 'LocationID','tpep_pickup_datetime':\
                                 'pickup_datetime'}, inplace = True)
        file[i] = pd.merge(file[i], taxi_zone[['LocationID','lon','lat']], how = 'left', on = 'LocationID')
        file[i].drop(['LocationID'], axis = 1, inplace = True)
        file[i].rename(columns = {'DOLocationID' : 'LocationID','lon':'pickup_longitude', 'lat':\
                                       'pickup_latitude'}, inplace = True)
        file[i] = pd.merge(file[i], taxi_zone[['LocationID','lon','lat']], how = 'left', on = 'LocationID')
        file[i].drop(['LocationID'], axis = 1, inplace = True)
        file[i].rename(columns = {'lon':'dropoff_longitude', 'lat':'dropoff_latitude'}, inplace = True)
    for i in range(len(file)):
        file[i] = file[i][(file[i]['pickup_longitude'] >= -74.242330) \
                                & (file[i]['pickup_longitude'] <= -73.717047) \
                                & (file[i]['pickup_latitude'] >= 40.560445) \
                                & (file[i]['pickup_latitude'] <= 40.908524) \
                                &(file[i]['dropoff_longitude'] >= -74.242330) \
                                & (file[i]['dropoff_longitude'] <= -73.717047) \
                                & (file[i]['dropoff_latitude'] >= 40.560445) \
                                & (file[i]['dropoff_latitude'] <= 40.908524)]
    file_res = pd.DataFrame()
    for i in range(len(file)):
        file_res = pd.concat([file_res,file[i].sample(round(year_num[n]/12+1))], axis = 0)
    file_res = file_res.sample(year_num[n])
    file_res['distance'] = file_res.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)
    
    return file_res

In [19]:
#define the function to get and clean data in 2010
def get_and_clean_data2(n):
    file = list(range(1,13))
    for i in range(len(file)):
        file[i] = pd.read_parquet(yellow_lst[i+12*n-24])
    for i in range(len(file)):
        file[i].drop(['vendor_id','dropoff_datetime','passenger_count',\
                            'trip_distance','rate_code','store_and_fwd_flag',\
                            'payment_type','fare_amount','surcharge','mta_tax',\
                            'tolls_amount','total_amount'], axis = 1, inplace = True)
    for i in range(len(file)):
        file[i]['pickup_datetime'] = pd.to_datetime(file[i]['pickup_datetime'])
    for i in range(len(file)):
        file[i] = file[i][(file[i]['pickup_longitude'] >= -74.242330) \
                                & (file[i]['pickup_longitude'] <= -73.717047) \
                                & (file[i]['pickup_latitude'] >= 40.560445) \
                                & (file[i]['pickup_latitude'] <= 40.908524) \
                                &(file[i]['dropoff_longitude'] >= -74.242330) \
                                & (file[i]['dropoff_longitude'] <= -73.717047) \
                                & (file[i]['dropoff_latitude'] >= 40.560445) \
                                & (file[i]['dropoff_latitude'] <= 40.908524)]
    file_res = pd.DataFrame()
    for i in range(len(file)):
        file_res = pd.concat([file_res,file[i].sample(round(year_num[n+5]/12+1))], axis = 0)
    file_res = file_res.sample(year_num[n+5])
    file_res['distance'] = file_res.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)
    
    return file_res

In [20]:
#define the function to get and clean data in 2009
def get_and_clean_data3(n):
    file = list(range(1,13))
    for i in range(len(file)):
        file[i] = pd.read_parquet(yellow_lst[i+12*n-12])
    for i in range(len(file)):
        file[i].drop(['vendor_name','Trip_Dropoff_DateTime','Passenger_Count',\
                            'Trip_Distance','Rate_Code','store_and_forward',\
                            'Payment_Type','Fare_Amt','surcharge','mta_tax',\
                            'Tolls_Amt','Total_Amt'], axis = 1, inplace = True)
    for i in range(len(file)):
        file[i].rename(columns = {'Trip_Pickup_DateTime' : 'pickup_datetime'\
                       ,'Start_Lon' : 'pickup_longitude', 'Start_Lat' : 'pickup_latitude',\
                       'End_Lon' : 'dropoff_longitude','End_Lat' : 'dropoff_latitude','Tip_Amt':'tip_amount'},\
                       inplace = True)
    for i in range(len(file)):
        file[i]['pickup_datetime'] = pd.to_datetime(file[i]['pickup_datetime'])
    for i in range(len(file)):
        file[i] = file[i][(file[i]['pickup_longitude'] >= -74.242330) \
                                & (file[i]['pickup_longitude'] <= -73.717047) \
                                & (file[i]['pickup_latitude'] >= 40.560445) \
                                & (file[i]['pickup_latitude'] <= 40.908524) \
                                &(file[i]['dropoff_longitude'] >= -74.242330) \
                                & (file[i]['dropoff_longitude'] <= -73.717047) \
                                & (file[i]['dropoff_latitude'] >= 40.560445) \
                                & (file[i]['dropoff_latitude'] <= 40.908524)]
    file_res = pd.DataFrame()
    for i in range(len(file)):
        file_res = pd.concat([file_res,file[i].sample(round(year_num[n+6]/12+1))], axis = 0)
    file_res = file_res.sample(year_num[n+6])
    file_res['distance'] = file_res.apply(lambda x: create_distance(x['pickup_latitude'],\
                                                                    x['pickup_longitude'],\
                                                                      x['dropoff_latitude'],\
                                                                   x['dropoff_longitude']), axis=1)
    
    return file_res


In [21]:
file2015_res = get_and_clean_data(0)

KeyboardInterrupt: 

In [ ]:
file2014_res = get_and_clean_data(1)

In [ ]:
file2013_res = get_and_clean_data(2)

In [ ]:
file2012_res = get_and_clean_data(3)

In [ ]:
file2011_res = get_and_clean_data(4)

In [ ]:
file2010_res = get_and_clean_data2(0)

In [ ]:
file2009_res = get_and_clean_data3(0)

In [ ]:
yellow_taxi = pd.DataFrame()
for i in [file2015_res, file2014_res, file2013_res,file2012_res,file2011_res,file2010_res,file2009_res]:
    yellow_taxi = pd.concat([yellow_taxi,i], axis = 0)

In [ ]:
yellow_taxi = yellow_taxi.reset_index()
yellow_taxi = yellow_taxi.drop(['index'], axis =1 )

In [ ]:
yellow_taxi.isnull().sum()

## Weather Data
1. Define functions for cleaning and processing weather data
2. Apply the functions for every year data to get yearly dataframes
3. Concatenate every year's data

In [ ]:
#This delete_str() function is for processing precipitation data
#Because precipitation is in string type and T denote trace amount,
#We use an relatively small number to replace the T in order for further number calculation.
#x[:4] because there are wrongly number with "0.003s", the s we assume is a typo
def delete_str(x):
    if x == None:
        return x
    elif x == 'T':
        return 0.0001
    elif type(x) == float:
        return x
    else:
        return x[:4]
    
#This reset_drop() function is for dataframe's column normalize
#Because we have to concatenate all the dataframes at last which need same column name and index
def reset_drop(x):
    x = x.reset_index()
    x = x.drop(['index'], axis = 1)
    return x

In [ ]:
#This Weather_transfer()function is used for processing yearly csv file
#into hourly weather and calculating daily weather
#Please note that calculating daily weather is specific for
#2009-2012's data, because these data do not have daily summery(Their "SOD" row count is much smaller than 365)
#For other 2013-2015's data, we only need to fetch the "SOD" report which is for daily
#(we have test that "SOD" rows for 2013 to 2015 all have 365 entries, which we assume is sufficient)
def Weather_transfer(csv_file):
    df = pd.read_csv(csv_file)
    
    df = df.loc[df['REPORT_TYPE'] != 'SOD  ']
    df = df.loc[df['REPORT_TYPE'] != 'SOM  ']
    
    df_h = df[['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    
    df_h['HourlyPrecipitation'] = df_h['HourlyPrecipitation'].apply(delete_str)

    df_h['DATE_parsed'] = pd.to_datetime(df_h['DATE'], format = "%Y-%m-%dT%H:%M:%S")

    #df_h['HourlyPrecipitation'] = df_h['HourlyPrecipitation'].replace(['T'], 1e-6)
    df_h['HourlyPrecipitation'] = df_h['HourlyPrecipitation'].astype(float)
    
    df_h['HourlyWindSpeed'] = df_h['HourlyWindSpeed'].fillna(0)
    df_h['HourlyPrecipitation'] = df_h['HourlyPrecipitation'].fillna(0)

    df_h['DATE_day'] = pd.to_datetime(df_h['DATE'], format = "%Y-%m-%d").dt.date
    group = df_h.groupby('DATE_day')
    df_d = group.mean()
    df_d.rename(columns={"HourlyPrecipitation": "DailyPrecipitation", "HourlyWindSpeed": "DailyAverageWindSpeed"}, inplace = True)
    df_d['DailyPrecipitation'] = df_d['DailyPrecipitation'].fillna(0)
    #df_h['HourlyPrecipitation'] = df_h['HourlyPrecipitation'].fillna(0)
    
    
    df_h['DATE'] = df_h['DATE_parsed']
    df_h.drop(['DATE_parsed','DATE_day'], axis = 1, inplace = True)
    
    df_d.reset_index(inplace=True)

    list_df = [df_h, df_d]
    return list_df

In [ ]:
#This get_daily() function is specific for year from 2013 to 2015
#As their SOD daily summery is compelete and reliable
def get_daily(csv_file):
    weather_df = pd.read_csv(csv_file)
    df_daily = weather_df.loc[weather_df['REPORT_TYPE'] == 'SOD  '][['DATE', 'DailyPrecipitation', 'DailyAverageWindSpeed']]
    df_daily['DailyPrecipitation'] = df_daily['DailyPrecipitation'].apply(delete_str)
    df_daily['DailyPrecipitation'] = df_daily['DailyPrecipitation'].astype(float)
    df_daily['DailyAverageWindSpeed'] = df_daily['DailyAverageWindSpeed'].astype(float)
    
    df_daily['DailyAverageWindSpeed'] = df_daily['DailyAverageWindSpeed'].fillna(0)
    
    df_daily['Date'] = pd.to_datetime(df_daily['DATE'], format = "%Y-%m-%dT%H:%M:%S").dt.date
    df_daily['DATE'] = df_daily['Date']
    df_daily = df_daily.drop(['Date'], axis = 1)
    df_daily.rename({'DATE' : 'DATE_day'}, axis = 1, inplace = True)
    #df_daily.set_index(['DATE_day'], inplace = True)
    df_daily.reindex(columns = ['DATE_day', 'DailyPrecipitation', 'DailyAverageWindSpeed'])
    
    df_daily.reset_index(inplace=True)
    df_daily = df_daily.drop(['index'], axis = 1)
    return df_daily

In [ ]:
#This get_sun function is for getting the sunrise and sunset time for everyday of 2009 to 2015
def get_sun(csv_file):
    weather_df_09 = pd.read_csv(csv_file)
    df = weather_df_09.loc[weather_df_09['REPORT_TYPE'] == 'SOD  '][['DATE', 'Sunrise', 'Sunset']]
    df['Sunrise'] = '0' + df['Sunrise'].astype(int).astype(str) 
    df['Sunset'] = df['Sunset'].astype(int).astype(str)
    df['DATE_day'] = pd.to_datetime(weather_df_09['DATE'], format = "%Y-%m-%d").dt.date
    df['DATE_Sunrise_str'] = df['DATE_day'].astype(str) + '0' + df['Sunrise'].astype(int).astype(str)
    df['DATE_Sunset_str'] = df['DATE_day'].astype(str) + df['Sunset'].astype(int).astype(str) 
    df['Sunrise_Time'] = pd.to_datetime(df['DATE_Sunrise_str'], format = "%Y-%m-%d%H%M")
    df['Sunset_Time'] = pd.to_datetime(df['DATE_Sunset_str'], format = "%Y-%m-%d%H%M")
    df_sun = df[['Sunrise_Time', 'Sunset_Time']]
    
    df_sun.reset_index(inplace=True)
    df_sun = df_sun.drop(['index'], axis = 1)
    return df_sun

In [ ]:
#Here we apply the functions to get a list of yearly generated dataframes
#Finally we stack the df one on another to get 2012-2015 data
#Note that Here is for hourly and daily weather
ls_09 = Weather_transfer('2009_weather.csv')
ls_10 = Weather_transfer('2010_weather.csv')
ls_11 = Weather_transfer('2011_weather.csv')
ls_12 = Weather_transfer('2012_weather.csv')
ls_13 = [Weather_transfer('2013_weather.csv')[0], get_daily('2013_weather.csv')]
ls_14 = [Weather_transfer('2014_weather.csv')[0], get_daily('2014_weather.csv')]
ls_15 = [Weather_transfer('2015_weather.csv')[0], get_daily('2015_weather.csv')]
name_lst = [ls_09, ls_10, ls_11, ls_12, ls_13, ls_14, ls_15]
weather_hour = pd.DataFrame()
weather_day = pd.DataFrame()
for i in range(7):
    weather_hour = pd.concat([weather_hour,name_lst[i][0]], axis = 0)
    weather_day = pd.concat([weather_day,name_lst[i][1]], axis = 0)

In [ ]:
#Same process as before but different function
#Note that Here is only for daily sunset and sunrise
sun_09 = get_sun('2009_weather.csv')
sun_10 = get_sun('2010_weather.csv')
sun_11 = get_sun('2011_weather.csv')
sun_12 = get_sun('2012_weather.csv')
sun_13 = get_sun('2013_weather.csv')
sun_14 = get_sun('2014_weather.csv')
sun_15 = get_sun('2015_weather.csv')
sun_lst = [sun_09, sun_10, sun_11, sun_12, sun_13, sun_14, sun_15]
sun_time = pd.DataFrame()
for i in range(7):
    sun_time = pd.concat([sun_time,sun_lst[i]],axis = 0)

In [ ]:
weather_hour = reset_drop(weather_hour)
weather_day = reset_drop(weather_day)
sun_time = reset_drop(sun_time)

## Part2 Create SQL database
create a database conytaining 5 tables (4 required table and 1 table for sunset and sunrise data)

In [ ]:
DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"
engine = db.create_engine(DATABASE_URL)

In [ ]:
COMMAND = ["""
DROP TABLE IF EXISTS taxi_trips
""","""
DROP TABLE IF EXISTS uber_trips""",
"""
DROP TABLE IF EXISTS hourly_weather""",
"""
DROP TABLE IF EXISTS daily_weater
"""]
for i in COMMAND:
    engine.execute(i)

In [ ]:
#Create our 5 tables
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NO EXITS hourly_weather
(
    id INTERGER PRIMARY KEY,
    DATE DATE,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NO EXITS daily_weather
(
    id INTERGER PRIMARY KEY,
    DATE_day DATE,
    DailyPrecipitation FLOAT,
    DailyAverageWindSpeed FLOAT,
)
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NO EXITS taxi_trips
(
    id INTERGER PRIMARY KEY,
    pickup_datetime DATETIME,
    tip_amount FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NO EXITS uber_trips
(
    id INTERGER PRIMARY KEY,
    pickup_datetime FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
)
"""

SUN_TIME_SCHEMA = """
CREATE TABLE IF NO EXITS sun_time
(
    id INTERGER PRIMARY KEY,
    Sunrise_Time DATETIME,
    Sunset_Time DATETIME
)
"""

In [ ]:
#Create the required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)
    f.write(SUN_TIME_SCHEMA)

In [ ]:
# Create the tables with the schema files
with engine.connect() as connection:
    pass

## Add data into database
add cleaned data to the table we create above

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    add = list(table_to_df_dict.items())
    for i in range(len(add)):
        add[i][1].to_sql(add[i][0],engine)
        print('added', add[i][0])

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": yellow_taxi,
    "uber_trips": uber_data,
    "hourly_weather": weather_hour,
    "daily_weather": weather_day,
    "sun_time":sun_time,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

In [ ]:
#check the data
pd.read_sql('daily_weather', engine)

In [ ]:
pd.read_sql('taxi_trips', engine)

## Part3 Understanding the Data

### Q1
For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.

In [ ]:
QUERY_1 = """
SELECT COUNT(*) AS trips, strftime('%H', pickup_datetime) AS hour
FROM taxi_trips
WHERE pickup_datetime <'2015-07-01 00:00:00'
GROUP BY hour
ORDER BY trips
"""
result1 = engine.execute(QUERY_1).fetchall()

### Q2
For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.